# Writing Formatted Output

## Overview

### Questions

* How do I display status information during a simulation?
* How can I log user-defined quantities?
* How can I write formatted output to a text file?

### Objectives

* Demonstrate **user-defined log quantities**.
* Explain the use of **Table** to display status information during a simulation run.
* Show that **Table** can write to a file.

## Boilerplate code

In [1]:
import hoomd
import datetime

In [2]:
import os
fn = os.path.join(os.getcwd(), 'log.txt')
![ -e "$fn" ] && rm "$fn"

## Define the Simulation

This tutorial executes the Lennard-Jones particle simulation from a previous tutorial. 
See [*Introducing Molecular Dyamics*](../01-Introducing-Molecular-Dynamics/00-index.ipynb) for a complete description of this code.

In [3]:
gpu = hoomd.device.GPU()
sim = hoomd.Simulation(device=gpu)
sim.create_state_from_gsd(filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Formatted output

The **Table** writer formats log quantities in human-readable text and writes them to `stdout` or a file.
**Table** only supports `scalar` and `string` quantities due to the limitations of this format.
This section shows you how to use **Table** to display status information during a simulation run.

## Add quantities to a Logger

The `categories` argument to **Logger** defines the **categories** that it will accept.

In [4]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])

Log the and simulation `timestep` and `tps` quantities:

In [5]:
logger.add(sim, quantities=['timestep', 'tps'])

You can also log user-defined quantities using functions, callable class instances, or class properties.
For example, this class computes the estimated time remaining:

In [6]:
class Status():
    def __init__(self, sim):
        self.sim = sim
    
    @property
    def seconds_remaining(self):
        try:
            return (self.sim.final_timestep - self.sim.timestep) / self.sim.tps
        except ZeroDivisionError:
            return 0        
    
    @property
    def etr(self):
        return str(datetime.timedelta(seconds=self.seconds_remaining))


Assign the loggable quantity using the tuple `(object, property_name, flag)`,  where `flag` is the string name of the flag for this quantity. (The tuple for callable objects would be `(callable, flag)`).

In [7]:
status = Status(sim)
logger[('Status', 'etr')] = (status, 'etr', 'string')

Represent the namespace of your user-defined quantity with a tuple of strings - `('Status', 'etr'`) above.
You can use any number of arbitrary strings in the tuple to name your quantity.

## Display quantities with Table

**Table** is a **Writer** that formats the quantities in a **Logger** into a human readable table.
Create one that triggers periodically:

In [8]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)

Add it to the simulation:

In [9]:
sim.operations.writers.append(table)

Run the simulation and see the output:

In [10]:
sim.run(100000)

Simulation.timestep  Simulation.tps     Status.etr   
       35000           5349.75680     0:00:17.757817 
       40000           6604.27207     0:00:13.627543 
       45000           8127.64945     0:00:10.458128 
       50000           9190.65778     0:00:08.704491 
       55000           9971.94580     0:00:07.521100 
       60000          10575.04052     0:00:06.619360 
       65000          11048.14889     0:00:05.883339 
       70000          11431.98314     0:00:05.248433 
       75000          11751.93056     0:00:04.680082 
       80000          12020.43226     0:00:04.159584 
       85000          12248.06934     0:00:03.674048 
       90000          12448.06219     0:00:03.213352 
       95000          12620.89793     0:00:02.773178 
      100000          12770.05168     0:00:02.349247 
      105000          12901.00958     0:00:01.937833 
      110000          13021.26823     0:00:01.535949 
      115000          13129.81976     0:00:01.142438 
      120000          13229.

Later in this notebook, you are going to create another **Table Writer**.
Remove `table` now to avoid confusing the two:

In [11]:
sim.operations.writers.remove(table)

## Save Table output to a file

**Table** writes to `stdout` by default.
It can write to a file (or any Python file-like object with `write` and `flush` methods) instead.

Open the file to write:

In [12]:
file = open('log.txt', 'x')

Create a **Table** **Writer** that outputs to this file and add it to the **Simulation**:

In [13]:
table_file = hoomd.write.Table(output=file,
                               trigger=hoomd.trigger.Periodic(period=5000),
                               logger=logger)
sim.operations.writers.append(table_file)

Run the simulation:

In [14]:
sim.run(100000)

You can read the file with standard tools:

In [15]:
!tail log.txt

      185000          15079.43580     0:00:02.984197 
      190000          15081.82908     0:00:02.652198 
      195000          15089.82881     0:00:02.319443 
      200000          15098.35316     0:00:01.986972 
      205000          15102.46657     0:00:01.655359 
      210000          15108.62957     0:00:01.323747 
      215000          15110.85543     0:00:00.992664 
      220000          15115.06809     0:00:00.661591 
      225000          15120.70428     0:00:00.330672 
      230000          15125.16959        0:00:00     


In this section, you have displayed **loggable quantities** during a simulation run and saved them to a text file.
This is the last section in the logging tutorial.

[Previous section](03-Storing-Particle-Shape.ipynb) / [Tutorial index](../README.md)